In [ ]:
!pip install surprise

     |████████████████████████████████| 11.8 MB 47 kB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1619428 sha256=c834b691aa410cf0e5c2445c5541d08f5b160029e8db513805c0eb7226a48bf0
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [ ]:
import numpy as np 
import pandas as pd 
from surprise import BaselineOnly
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise import SVD
from surprise import accuracy
from surprise.model_selection import KFold
from surprise import KNNBasic,KNNWithMeans
from surprise import accuracy
from surprise.model_selection import PredefinedKFold
from surprise.model_selection import train_test_split
from collections import defaultdict
import io
from surprise.model_selection import GridSearchCV
import csv
import os
import time

In [ ]:
df_wisata = pd.read_csv('hasilrating.csv')
df_wisata.head()

,id_wisata,id_user,rating
0,1,1118914,5
1,1,1925512,2
2,1,5618425,3
3,1,9144181,2
4,1,13212611,3


In [ ]:
df_wisata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2866 entries, 0 to 2865
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   id_wisata  2866 non-null   int64
 1   id_user    2866 non-null   int64
 2   rating     2866 non-null   int64
dtypes: int64(3)
memory usage: 67.3 KB


In [ ]:
counts = df_wisata['id_wisata'].value_counts()
df_wisata= df_wisata[df_wisata['id_wisata'].isin(counts[counts >= 10].index)]

In [ ]:
df_wisata.shape

(2859, 3)

In [ ]:
counts1 = df_wisata['id_user'].value_counts()
#print(counts1)
df_wisata= df_wisata[df_wisata['id_user'].isin(counts1[counts1 > 3].index)]

In [ ]:
df_wisata.describe()

,id_wisata,id_user,rating
count,207.000000,2.070000e+02,207.000000
mean,17.159420,4.974511e+06,2.637681
std,8.529145,1.212039e+07,1.474420
min,2.000000,7.540000e+02,0.000000
25%,9.000000,4.201600e+04,2.000000
50%,20.000000,8.514410e+05,3.000000
75%,23.000000,4.114916e+06,4.000000
max,29.000000,6.188202e+07,5.000000


In [ ]:
df1 = df_wisata[['id_wisata','id_user','rating']]
df1.head()
print(df1.nunique())

id_wisata    27
id_user      37
rating        6
dtype: int64


In [ ]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df1[['id_wisata','id_user','rating']], reader)

data1 = data

In [ ]:
raw_ratings = data.raw_ratings

In [ ]:
raw_ratings

[(2, 118725, 2.0, None),
 (2, 1892611, 3.0, None),
 (2, 61882019, 2.0, None),
 (3, 3433, 5.0, None),
 (3, 4931, 0.0, None),
 (3, 9819, 0.0, None),
 (3, 12948, 3.0, None),
 (3, 105661, 5.0, None),
 (3, 118725, 0.0, None),
 (3, 125123, 4.0, None),
 (3, 142118, 0.0, None),
 (3, 201891, 0.0, None),
 (3, 696919, 0.0, None),
 (3, 923114, 5.0, None),
 (3, 1133820, 0.0, None),
 (3, 1311891, 5.0, None),
 (3, 1892611, 0.0, None),
 (3, 1892611, 0.0, None),
 (3, 1988127, 4.0, None),
 (3, 2939155, 0.0, None),
 (3, 5382514, 5.0, None),
 (3, 7239124, 0.0, None),
 (3, 7517577, 4.0, None),
 (3, 7517577, 0.0, None),
 (3, 12111914, 0.0, None),
 (3, 12111914, 0.0, None),
 (3, 61882019, 2.0, None),
 (4, 75451, 3.0, None),
 (4, 125123, 2.0, None),
 (4, 546155, 5.0, None),
 (4, 4114916, 4.0, None),
 (4, 7517577, 3.0, None),
 (4, 49525111, 3.0, None),
 (5, 754, 3.0, None),
 (5, 13118, 3.0, None),
 (5, 5382514, 5.0, None),
 (5, 61882019, 3.0, None),
 (6, 3433, 5.0, None),
 (6, 75451, 2.0, None),
 (6, 851441, 5

In [ ]:
hasil = df_wisata.groupby('id_wisata')['rating'].sum()

In [ ]:
hasil

id_wisata
1       30
2       93
3      574
4      156
5      236
6      127
7      214
8       47
9      208
10      75
11      52
12      87
13      23
14      45
15      54
16      58
17      25
18      41
19     850
20     993
21    1125
22     119
23    1164
24     137
25     359
26     186
27      98
28     437
29     366
Name: rating, dtype: int64

In [ ]:
hasil.to_csv('hasilsvd.csv',index=False)

In [ ]:
#raw_ratings.to_csv('hasilsvd.csv',index=False)

In [ ]:
threshold = int(.8 * len(raw_ratings))

In [ ]:
A_raw_ratings = raw_ratings[:threshold]
B_raw_ratings = raw_ratings[threshold:]

trainset, testset = train_test_split(data, test_size=.20)

In [ ]:
data.raw_ratings = A_raw_ratings  # data is now the set A
data1.raw_ratings = B_raw_ratings

In [ ]:
start = time.time()
param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005,0.1],
              'reg_all': [0.4, 0.6], 'n_factors':[100,500]}
grid_search = GridSearchCV(SVD, param_grid, measures=['rmse','mae'], cv=3)
grid_search.fit(data)
end = time.time()
print('best RMSE score')
print(grid_search.best_score['rmse'])
print("Time : ",end-start)

best RMSE score
1.1750992426917082
Time :  0.21423554420471191


In [ ]:
print('combination of parameters that gave the best RMSE score')
print(grid_search.best_params['rmse'])
algo = grid_search.best_estimator['rmse']

combination of parameters that gave the best RMSE score
{'n_epochs': 10, 'lr_all': 0.002, 'reg_all': 0.6, 'n_factors': 500}


In [ ]:
trainset = data.build_full_trainset()
algo.fit(trainset)

# Compute biased accuracy on A
testset= trainset.build_testset()
predictions = algo.test(testset)
print('Biased accuracy on A,', end='   ')
accuracy.rmse(predictions,verbose=True)
print('len(predictions)')
print(len(predictions))

Biased accuracy on A,   RMSE: 0.9444
len(predictions)
42


In [ ]:
# Compute unbiased accuracy on B
testset = data.construct_testset(B_raw_ratings)  # testset is now the set B
predictions = algo.test(testset)
print('Unbiased accuracy on B,', end=' ')
accuracy.rmse(predictions)
print('len(predictions)')
print(len(predictions))

Unbiased accuracy on B, RMSE: 0.9444
len(predictions)
42


In [ ]:
# define a cross-validation iterator
kf = KFold(n_splits=7)
#algo = SVD(n_factors=500, n_epochs=5, lr_all=0.1)

for trainset, testset in kf.split(data1):
    # train and test algorithm.
    algo.fit(trainset)
    
    predictions = algo.test(testset)
    
    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions, verbose=True)

RMSE: 1.6923
RMSE: 1.4034
RMSE: 0.4886
RMSE: 1.3842
RMSE: 0.7703
RMSE: 1.7135
RMSE: 0.7732
